# Julia - Day 3

https://adventofcode.com/2021/day/3

## Data

In [1]:
using CSV
using DataFrames

example = CSV.read(
    IOBuffer(
    """00100
    11110
    10110
    10111
    10101
    01111
    00111
    11100
    10000
    11001
    00010
    01010"""),
    DataFrame,
    header = false,
    types = Dict(1 => String)
)
rename!(example, [:binary_diagnostic])
replace!(example[!, :binary_diagnostic], r"\s" => "") #remove whitespaces

input = CSV.read("Input/3_1.txt", DataFrame, header  = false, types = Dict(1 => String))
rename!(input, [:binary_diagnostic])

,binary_diagnostic
,String
1,000110000001
2,011011001101
3,001101100111
4,001101011001
5,110111011101
6,110011101010
7,111101010001
8,010100111101
9,011000011000


In [30]:

example

,binary_diagnostic
,String
1,00100
2,11110
3,10110
4,10111
5,10101
6,01111
7,00111
8,11100
9,10000


In [12]:
first(input, 5)

,binary_diagnostic
,String
1,000110000001
2,011011001101
3,001101100111
4,001101011001
5,110111011101


## Part 1

### Function

In [79]:
function power_consumption_data(binary_diag)
    string_len = length(binary_diag[1, :binary_diagnostic])

    #split each diagnostic by digit
    split_strings = split.(binary_diag[!, :binary_diagnostic], "")
    colnames = Symbol.(string.("L", 1:string_len))

    for (i, n) in enumerate(colnames)
        # create a column for each ith letter in :binary_diagnostic
        binary_diag[!, n] = getindex.(split_strings, i)
    end

    binary_diag = stack(binary_diag, Not(:binary_diagnostic))

    # count number of 0 and 1 for each position
    binary_diag = groupby(binary_diag, [:variable, :value])
    binary_diag = combine(
        binary_diag,
        nrow        
    )

    # gamma rate and epsilon rate
    binary_diag = groupby(binary_diag, :variable)
    binary_diag = combine(
        binary_diag,
        [:value, :nrow] => (
            (value, nrow) -> (
                gamma_rate = value[argmax(nrow)],
                epsilon_rate = value[argmin(nrow)]
            ) 
        ) => AsTable
    )
 
    result = DataFrame(
        rate = ["gamma", "epsilon"],
        binary = [reduce(string, binary_diag[!, "gamma_rate"]), reduce(string, binary_diag[!, "epsilon_rate"])]
    )
    result[!, :decimal] = parse.(Int, result[!, :binary], base = 2)

    return result

end


function power_consumption(power_consumption_data) 
    return power_cons_data[power_cons_data.rate .== "gamma", :decimal] .* power_cons_data[power_cons_data.rate .== "epsilon", :decimal]
end

power_consumption (generic function with 1 method)

### Result

In [81]:
#Example
power_cons_data = power_consumption_data(example)
result = power_consumption(power_cons_data)
println(
    string("Example : ", result)
)

#Input
power_cons_data = power_consumption_data(input)
result = power_consumption(power_cons_data)
println(
    string("Input : ", result)
)


Example : [198]
Input : [3429254]


## Part 2

### Function

In [14]:
function oxygen_generator_rating(binary_diag)

    string_len = length(binary_diag[1, :binary_diagnostic])

    #split each diagnostic by digit
    split_strings = split.(binary_diag[!, :binary_diagnostic], "")
    colnames = Symbol.(string.("L", 1:string_len))

    for (i, n) in enumerate(colnames)
        # create a column for each ith letter in :binary_diagnostic
        binary_diag[!, n] = getindex.(split_strings, i)
    end

    
    # Oxygen cryteria, iter over each col until only one number left
    n_number = nrow(binary_diag)

    i = 1

    while n_number > 1

        # count zeros and ones for bit i
        bit_criteria = binary_diag[!, colnames[i]]
        zeros = count(bit_criteria .== "0")
        ones = count(bit_criteria .== "1")

        # keep bit starting with 0 if more zero than one
        if zeros > ones
            bit_criteria = "0"
        elseif ones >= zeros
            bit_criteria = "1"
        end

        #filter with criteria
        binary_diag = filter(colnames[i] => ==(bit_criteria), binary_diag)

        n_number = nrow(binary_diag)
        i += 1

    end


    result = DataFrame(
        rate = "oxygen_generator",
        binary = binary_diag[!, :binary_diagnostic]
    )
    result[!, :decimal] = parse.(Int, result[!, :binary], base = 2)

    return result

end


function CO2_scrubber_rating(binary_diag)

    string_len = length(binary_diag[1, :binary_diagnostic])

    #split each diagnostic by digit
    split_strings = split.(binary_diag[!, :binary_diagnostic], "")
    colnames = Symbol.(string.("L", 1:string_len))

    for (i, n) in enumerate(colnames)
        # create a column for each ith letter in :binary_diagnostic
        binary_diag[!, n] = getindex.(split_strings, i)
    end

    
    # Oxygen cryteria, iter over each col until only one number left
    n_number = nrow(binary_diag)

    i = 1

    while n_number > 1

        # count zeros and ones for bit i
        bit_criteria = binary_diag[!, colnames[i]]
        zeros = count(bit_criteria .== "0")
        ones = count(bit_criteria .== "1")

        # keep bit starting with 0 if more zero than one
        if ones >= zeros
            bit_criteria = "0"
        elseif zeros > ones
            bit_criteria = "1"
        end

        #filter with criteria
        binary_diag = filter(colnames[i] => ==(bit_criteria), binary_diag)

        n_number = nrow(binary_diag)
        i += 1

    end

    result = DataFrame(
        rate = "CO2_scrubber",
        binary = binary_diag[!, :binary_diagnostic]
    )
    result[!, :decimal] = parse.(Int, result[!, :binary], base = 2)

    return result

end


function life_support_rating(oxygen_generator_data, CO2_scrubber_data)

    result = DataFrame(
        rate = "life_support",
        decimal = oxygen_generator_data[!, :decimal] .* CO2_scrubber_data[!, :decimal]
    )
    result[!, :binary] = [reduce(string, reverse(digits(first(result[!, :decimal]), base=2)))]

    return result
    
end

life_support_rating (generic function with 1 method)

### Result

In [18]:
#Example
CO2_scrubber_data = CO2_scrubber_rating(example)
oxygen_generator_data = oxygen_generator_rating(example)
life_support_data = life_support_rating(oxygen_generator_data, CO2_scrubber_data)

result = life_support_data[!, :decimal]

println(
    string("Example : ", result)
)
#Input
CO2_scrubber_data = CO2_scrubber_rating(input)
oxygen_generator_data = oxygen_generator_rating(input)
life_support_data = life_support_rating(oxygen_generator_data, CO2_scrubber_data)

result = life_support_data[!, :decimal]

println(
    string("Input : ", result)
)


Example : [230]
Input : [5410338]
